<a href="https://colab.research.google.com/github/JulioCezar1111/Projeto29/blob/main/Deep_learning_educacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prever o abandono ou o sucesso acadêmico**

# **Importar as bibliotecas**

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf # para redes neurais
import sklearn as sk

# **Importar para dataframe(pandas)**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dropout_Academic Success - Sheet1.csv')



print(df.Target.unique()) # informa as opçoes na coluna

from sklearn.preprocessing import LabelEncoder # converter categoria em numero

cols = ['Target'] #unique pra descobrir quantos rótulos unicos
le = LabelEncoder()
df[cols] = df[cols].apply(le.fit_transform) # fit_transform aprende e transforma

df.head(10)

**Maritial status:** Estado civil

**Application mode:** Modo de aplicativo

**Application order:** Pedido de inscrição

**Course:** Curso

**Daytime/evening attendance:** Perido

# **Tratar dataframe**

In [ ]:
df.info()

# **Separar dataframe em variveis independentes(X e y)**

In [ ]:
y = df['Target']
X = df.drop('Target', axis = 1)

display(X)

# **Dividar dataframe de treino e teste**

In [7]:
from sklearn.model_selection import train_test_split # para dividir o dataframe

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# **Normalizar os dados**

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train

# **Arquitetura da Rede Neural(MLP)**

**Bibliotecas**

In [97]:
from sklearn.model_selection import GridSearchCV # Auxiliar para achar os melhores parametros
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier # Para fazer a comunicação entre keras e gridsearch 
from tensorflow.keras.layers import Dropout, BatchNormalization

**Metodo da arquitetura da Rede Neural**

In [123]:
from keras.backend import dropout
def create_model(optimizer='Adam'):

    ann = Sequential()
    ann.add(Dense(units = 32, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 64, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 128, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 255, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())
    ann.add(Dropout(0.5))

    ann.add(Dense(units = 3, activation='softmax'))

    ann.compile(optimizer=optimizer, 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [124]:
model = KerasClassifier(build_fn=create_model, verbose = 2 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


**define the grid search parameters part 1**

In [ ]:
optimizer = ['Adam']
batch_size = [16, 32]
epochs = [30, 50, 70]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

**Parametros**

In [112]:
batch_size = grid_result.best_params_['batch_size']
epochs = grid_result.best_params_['epochs']
optimizer = grid_result.best_params_['optimizer']

print(batch_size, epochs, optimizer)

32 30 Adam


**Metodo da rede neural otimizador**

In [79]:
def create_model_Adam(n1 = 8, n2 = 16, n3= 32, learn_rate = 0.01):

    ann = Sequential()
    ann.add(Dense(units = n1, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n2, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n3, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())
    ann.add(Dropout(0.5))

    ann.add(Dense(units = 3, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = learn_rate)

    ann.compile(optimizer=optimizer, 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [80]:
model = KerasClassifier(build_fn = create_model_Adam, epochs = epochs, batch_size = batch_size, verbose = 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


**Grid search part2**

In [ ]:
learn_rate = [0.001, 0.01, 0.1]
n1 = [8, 16]
n2 = [16, 32]
n3 = [32, 64]

param_grid = dict(n1 = n1, n2 = n2, n3 = n3, learn_rate = learn_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = None, cv = 5)
grid_result = grid.fit(X_train, y_train)

# **Resultado**

In [113]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.736645 using {'batch_size': 32, 'epochs': 30, 'optimizer': 'Adam'}
0.736645 (0.010224) with: {'batch_size': 32, 'epochs': 30, 'optimizer': 'Adam'}


# **Testar Modelo**

In [114]:
y_pred = grid_result.predict(X_test)



In [115]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred) 
#vn, fp(rede disse que é 1, mas é 0), fn (rede disse que é 0, mas é 1), vp
cm

array([[196,  40,  39],
       [ 49,  55,  54],
       [ 10,  33, 409]])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print('Acurácia da Rede: ', accuracy_score(y_test, y_pred).round(2))
print(classification_report(y_test, y_pred))